# Utilisation de l'API GitHub

In [121]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from threading import Thread, RLock
import sys
import time

# Creation de la classe thread et des fonctions utilisées

In [137]:

verrou = RLock()

        
def star_count_t(page):
    #for a given page, return the number of stars, and the number of repos
    stars = [(repos["name"],repos["stargazers_count"]) for repos in page]
    return stars

def all_page_count_t(user):
    i=1
    token = "807b2aa256e504d205bbe65d2c968c1ab5a86a69"
    headers = {'Authorization': 'token {}'.format(token)}
    req = requests.get(f"https://api.github.com/users/{user}/repos?page=1?per_page=1000000",headers=headers)
    a=True
    while True:
        page = req.json()
        if req.status_code == 200:
            all_repos = []
            all_repos+=star_count_t(page)
            break
        else:
            time.sleep(1)
    return dict(all_repos)

def stars_users_t(user,stars):
    #Creation d'un dictionnaire résumant le nombre de stars et de repos pour chaque user
    repos_user = all_page_count_t(user)
    with verrou:
        stars["user"].append(user)
        stars["#_repos"].append(len(repos_user.keys()))
        stars["tot_stars"].append(sum(repos_user.values()))
        return stars

class git_crawl_user(Thread):
    

    """Thread chargé simplement d'afficher une lettre dans la console."""


    def __init__(self, user, stars):

        Thread.__init__(self)

        self.user = user
        
        self.stars = stars

        
    

    def run(self):
        self.stars=stars_users_t(self.user,self.stars)

## On crawl le top 256 contributors 

In [139]:
#Initiatilisation du dictionnaire principale
stars = {}
stars["user"]=[]
stars["#_repos"]=[]
stars["tot_stars"]=[]

t1 = time.time()
#Calcul de la liste des utilisateurs les plus actifs
list_users=get_top_users()

threads = []
#Creation des threads
i=0
for user in list_users:
    thread=git_crawl_user(user,stars)
    thread.start()
    time.sleep(1)
    i+=1
    if i == 25:
        print(round(len(stars["user"])/len(list_users)*100,0))
        i=0

t2 = time.time()
print(f"Tache terminé en {(t2-t1)}min")






    

10.0
20.0
29.0
39.0
49.0
59.0
68.0
78.0
88.0
98.0
Tache terminé en 0.2595426139831543s


In [148]:
rdf=pd.DataFrame(stars)
df["moyenne"]=round(df["tot_stars"]/df["#_repos"])
df.sort_values("moyenne", ascending=False)

,user,#_repos,tot_stars,moyenne
224,bailicangdu,13,45474,3498.0
15,kamranahmedse,44,99906,2271.0
140,vinta,32,58507,1828.0
115,mrdoob,35,55862,1596.0
193,IanLunn,18,25223,1401.0
42,sebastianbergmann,60,82984,1383.0
229,junyanz,14,19195,1371.0
150,nolimits4web,14,18722,1337.0
195,ruanyf,53,66495,1255.0
108,JakeWharton,100,107229,1072.0


In [150]:
"thejameskyle" in list_users

True